In [1]:
# !pip3 install openpyxl
import pandas as pd
import re


In [2]:
df = pd.read_excel(r'UPR_cycle2.xlsx')

In [3]:
df.index+=1
df.rename(columns={"Reccomending Body": "Recommending Body", "UPR Reccomending States": "UPR Recommending States"}, inplace=True)

# Data Cleaning

##### - Check unique number of text & annotation id to establish primary key
##### - Both have 625 unique entries which is the same as the total database entries
##### - Normalise by removing one of these, two identifiers is uneccesary we are not gaining anymore information

In [4]:
text_count = df.Text.unique().size
print(text_count)

625


In [5]:
annotation_id_count = df["OHCHR Annotation Id"].unique().size
print(annotation_id_count)

625


##### Columns to be dropped
##### Annotation ID, Recommending body, Text, Affected persons, Sdgs, Document publication date, session, date
##### check all entries are recommendation by seeing if text starts with number, then drop this column. 

In [6]:
drop_columns = ["OHCHR Annotation Id","Recommending Body", "Document Publication Date", "Sdgs", "UPR Session", "Date of publication on UHRI", "Affected Persons"]

In [7]:
df.drop(drop_columns, inplace=True, axis=1)

In [8]:
def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

In [9]:
df = df[df['Text'].apply(lambda x: has_numbers(x))]
df.drop(['Type'], inplace=True, axis=1)

In [10]:
df.index = df.index.set_names(['Recommendation_id'])

#### Standardize the naming convention, by removing hyphens and spaces, of all data entries.

In [11]:
def remove_leading_char(data_entry):
    """"Clean the naming convention for data entries of all columns"""
    clean_aff = re.sub(r'^[^A-Z]*', '', data_entry)
    if clean_aff:
        return clean_aff

In [12]:
def add_empty_cell_string(df):
    """Replacing all empty cells with default string 'Missing data'"""
    df.fillna('Missing data', inplace=True)
    return df 

In [13]:
df2 = add_empty_cell_string(df)

In [14]:
df2['Themes'] = df2['Themes'].str.split('\n')
df2 = df2.explode('Themes')

In [15]:
df3 = df2.applymap(lambda x: remove_leading_char(str(x)))

In [16]:
df4 = df3
df4['Recommending Regions'] = df4['Recommending Regions'].str.split('\n')
df4['UPR Recommending States'] = df4['UPR Recommending States'].str.split('\n')
df4 = df4.explode(['UPR Recommending States', 'Recommending Regions'])
df4 = df4.applymap(lambda x: remove_leading_char(str(x)))

In [17]:
Recommendations_df = df[['Text', 'UPR Position']].applymap(lambda x: remove_leading_char(str(x)))
Recommendations_df= Recommendations_df.reset_index(level=0)

In [18]:
Country_to_recommendations = df4[['Countries Concerned','UPR Recommending States']]
Country_to_recommendations.index = Country_to_recommendations.index.set_names(['Recommendation_id'])
Country_to_recommendations = Country_to_recommendations.reset_index(level=0).drop_duplicates(subset='Recommendation_id', keep='first')

In [19]:
c1 = df4[['Countries Concerned','Regions Concerned']].rename(columns={"Countries Concerned":"Country name", "Regions Concerned": "Regions"})
c2 = df4[['UPR Recommending States', 'Recommending Regions']].rename(columns={"UPR Recommending States":"Country name", "Recommending Regions": "Regions"})
Countries_df = pd.concat([c1,c2], ignore_index=True).drop_duplicates(subset=['Country name'], keep='first')
Countries_df = Countries_df.reset_index(drop='True')
Countries_df.index = Countries_df.index.set_names(['Country id'])
Countries_df.index+=1
Countries_df = Countries_df.reset_index()

In [20]:
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='Countries Concerned', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['Countries Concerned','Country name']).rename(columns={"Country id": "Countries Concerned"})
Country_to_recommendations = Country_to_recommendations.merge(Countries_df[["Country name","Country id"]], left_on='UPR Recommending States', right_on='Country name')
Country_to_recommendations = Country_to_recommendations.drop(columns=['UPR Recommending States','Country name']).rename(columns={"Country id": "UPR Recommending States"})

In [21]:
Regions_df = pd.DataFrame(Countries_df['Regions'].unique(), columns=['Regions'])
Regions_df.index = Regions_df.index.set_names(['Region id'])
Regions_df.index+=1
Regions_df = Regions_df.reset_index(level=0)

In [22]:
Countries_df = Countries_df.merge(Regions_df, left_on="Regions", right_on="Regions")
Countries_df = Countries_df.drop(columns=["Regions"])

In [23]:
Recommendation_to_theme = df4[['Themes']]
Recommendation_to_theme.index = Recommendation_to_theme.index.set_names(['Recommendation id'])
Recommendation_to_theme = Recommendation_to_theme.reset_index(level=0)

In [33]:
Themes_df = pd.DataFrame(df4['Themes'].unique(), columns=['Themes'])
Themes_df.index = Themes_df.index.set_names(['Theme id'])
Themes_df.index+=1
Themes_df = Themes_df.reset_index(level=0)


In [25]:
Recommendation_to_theme = Recommendation_to_theme.merge(Themes_df, left_on='Themes', right_on='Themes')
Recommendation_to_theme = Recommendation_to_theme.drop(columns=['Themes']).rename(columns={'Theme id':'Themes'})

### Normalized Dataframes

In [26]:
Recommendations_df

,Recommendation_id,Text,UPR Position
0,1,Promptly ratify the Optional Protocol to the C...,Noted
1,2,Ensure full and equal access to modern contrac...,Supported
2,3,Protect the rights of indigenous peoples throu...,Supported
3,4,Continue enhancing the school infrastructure f...,Supported
4,5,Continue implementing mechanisms for the prior...,Supported
...,...,...,...
620,621,Adopt legislation to guarantee the fulfilment ...,Supported
621,622,Establish clear consultation procedures in ord...,Noted
622,623,Consider ratifying ILO Convention N° 169 ( Nor...,Supported
623,624,Extend an invitation to the Working Group on e...,Supported


In [27]:
Country_to_recommendations

,Recommendation_id,Countries Concerned,UPR Recommending States
0,1,1,70
1,29,11,70
2,154,28,70
3,164,30,70
4,206,33,70
...,...,...,...
620,523,65,147
621,533,67,148
622,590,73,148
623,548,70,68


In [28]:
Countries_df

,Country id,Country name,Region id
0,1,Uganda,1
1,6,United Republic of Tanzania,1
2,7,Eswatini,1
3,15,Namibia,1
4,16,Sierra Leone,1
...,...,...,...
144,132,Saudi Arabia,5
145,133,Viet Nam,5
146,135,State of Palestine*,5
147,148,Oman,5


In [29]:
Regions_df

,Region id,Regions
0,1,Africa
1,2,Latin America and the Caribbean
2,3,Eastern Europe
3,4,Western Europe & Others
4,5,Asia-Pacific
5,6,Missing data


In [30]:
Recommendation_to_theme

,Recommendation id,Themes
0,1,0
1,9,0
2,11,0
3,16,0
4,35,0
...,...,...
1534,525,64
1535,543,65
1536,544,65
1537,589,66


In [31]:
Themes_df

,Theme id,Themes
1,0,Labour rights and right to work
2,1,Ratification of & accession to international i...
3,2,Prohibition of torture & ill-treatment (includ...
4,3,Sexual & reproductive health and rights
5,4,Right to participate in public affairs & right...
...,...,...
64,63,Good governance & corruption
65,64,Private life & privacy
66,65,Right to be recognized as a person before the law
67,66,Trade union rights
